### Mateusz Zacharecki, 05.01.2024
# Report for assignment 2 

## Imports

In [1]:
import numpy as np
import pandas as pd
import os, os.path
import sqlite3
import tempfile

In [2]:
from Zacharecki_Mateusz_assignment_2 import *

## Reading the data

In [3]:
Posts = pd.read_csv("Posts.csv.gz", compression = "gzip")
Users = pd.read_csv("Users.csv.gz", compression = "gzip")
Comments = pd.read_csv("Comments.csv.gz", compression = "gzip")
PostLinks = pd.read_csv("PostLinks.csv.gz", compression = "gzip")

## Creation of the database

In [4]:
# path to database file
baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza): # if this file already exists...
    os.remove(baza) # ...we will remove it
conn = sqlite3.connect(baza) # create the connection

# import the data frame into the database
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)

29019

## Results of comparing the equivalence of solutions

### Solution 1

In [5]:
pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn).equals(solution_1(Posts, Users))

True

### Solution 2

In [6]:
pd.read_sql_query("""
SELECT Posts.Title, RelatedTab.NumLinks
FROM
    (
        SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
        FROM PostLinks
        GROUP BY RelatedPostId
    ) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
""", conn).equals(solution_2(Posts, PostLinks))

True

### Solution 3

In [7]:
pd.read_sql_query("""
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
        SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
               CmtTotScr.CommentsTotalScore
        FROM (
                SELECT PostId, SUM(Score) AS CommentsTotalScore
                FROM Comments
                GROUP BY PostId
             ) AS CmtTotScr
        JOIN Posts ON Posts.Id = CmtTotScr.PostId
        WHERE Posts.PostTypeId=1
     ) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10
""", conn).equals(solution_3(Posts, Users, Comments))

True

### Solution 4

In [8]:
pd.read_sql_query("""
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
        SELECT *
        FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
             ) AS Answers
        JOIN
             (
                SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 1
                GROUP BY OwnerUserId
             ) AS Questions
        ON Answers.OwnerUserId = Questions.OwnerUserId
        WHERE AnswersNumber > QuestionsNumber
        ORDER BY AnswersNumber DESC
        LIMIT 5
     ) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
""", conn).equals(solution_4(Posts, Users))

True

### Solution 5

In [9]:
pd.read_sql_query("""
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
     ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
     ) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
""", conn).equals(solution_5(Posts))

True

In [10]:
conn.close()

## Summary

Gladly, we get all solutions equivalent to SQL query results. Both SQL and Python code execution time seem to work pretty fast, especially read_sql_query function from Pandas seems to work much better than sqldf function in R. Although in this assignment it was necessary to take care of indices due to how equals functions works, working with Pandas was quite a pleasure for me.